#### Importing Packages

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

#### Quantization configuration - 4 bits

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

#### Model loading

In [ ]:
token = ''

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config, token=token)
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

#### Model pipeline

In [ ]:
pipeline2 = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=6000,
        temperature=0.1,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.prompts import ChatPromptTemplate
llm_2 = HuggingFacePipeline(pipeline=pipeline2)

#### Answer generation

In [ ]:
question = input("Enter the question: ")
student_answer = input("Enter the student answer: ")
rubric = input("Enter the rubric: ")

Enter the question: Apply bioinformatics methods to analyze genomic data and identify genetic markers associated with complex diseases, facilitating personalized medicine and treatment strategies.
Enter the student answer: Bioinformatics utilizes various tools like sequence alignment, genome mapping, and variant analysis to analyze genetic information and uncover genetic links to complex diseases. By comparing the genetic makeup of individuals with and without the disease, these techniques identify genetic variations associated with increased risk or response to treatment. This knowledge facilitates the development of personalized medicine approaches, where treatment plans are customized based on an individual's unique genetic makeup. This personalization ultimately aims to improve treatment effectiveness and patient well-being.
Enter the rubric: Analyze genomic data to identify genetic markers - 1 Mark. Pinpoint genetic variations associated with diseases - 1 Mark. Develop personalize

In [ ]:
prompt1 = ChatPromptTemplate.from_template("""[INST] You are a helpful, respectful and a good assistant. You will follow my words. Answer the question with the help of rubrics (rubrics contains the important points that should be in the answer). Do not mention anything about marks in the answer. Generate only the answer, not anything else. Answer only in 200 words.

Rubrics:
{rubrics}
------------- rubric ends here --------------

Question: {question} [/INST]""")
chain1 = LLMChain(prompt=prompt1, llm=llm_2)
response = chain1.invoke({"rubrics": rubric, "question": question})
inst_index = response['text'].find("[/INST]")
res = response['text'][inst_index + len("[/INST]"):]
res

' Bioinformatics methods can be applied to analyze genomic data and identify genetic markers associated with complex diseases. This process involves the use of various computational tools and statistical analyses to identify genetic variations that are linked to specific diseases. For instance, next-generation sequencing data can be analyzed to identify single nucleotide polymorphisms (SNPs) and other genetic variations that are associated with diseases.\n\nOnce these genetic markers have been identified, they can be used to develop personalized medicine approaches. This involves tailoring treatment strategies to individual genetic profiles, allowing for more effective and efficient treatment. For example, certain medications may be more effective for individuals with specific genetic variations, while others may be less effective or even contraindicated.\n\nMoreover, genomic analysis can be used to optimize patient outcomes by identifying genetic factors that influence treatment effic

#### Answer evaluation (with rubric)

In [ ]:
prompt2 = ChatPromptTemplate.from_template("""
[INST] I am an objective and thorough and extremely strict assistant dedicated to rigorously evaluating student responses based solely on the provided question and rubric. I will only see what is explicitly provided to me. I shall do everything as it's said. I must not generate new answers or engage in emotional responses. I will be extremely strict in evaluation and extremely strict in providing marks. I will not be influenced in marking by implicitly implying anything from the student answer. I will mark the student answer only based on the given scoring criteria, and nothing else. I will provide marks only as per the criteria.

**QUESTION:** {question}
---- The question ends here -----

**RUBRIC:** {rubric}
------ The rubric ends here -----

**STUDENT ANSWER:** {student_answer}
------The student answer ends here-----

#Only follow these instructions for evaluation.
**Scoring:**

* Dont implicitly assume any criterion of rubric with the answer.
* Award 0 for the criterion with only implicit mentions.
* If a particular point met a criterion, do not check the same point of the answer with other criteria.
* Do not award marks for implicit points as they are not explicit to be marked on.
* Award 0 for the criterion, for any implicit match of the answer with the criterion.
* Award 0 for the entire answer, if the entire answer is just a rephrased version of the given question, and you are extremely strictly ordered to not continue evaluating on other criteria. Stop checking other criteria.
* Award 0 marks for the criterion, if there is no string match of that criterion in student answer.
* Award 0 marks for the criterion, if the criteria is only implicitly mentioned in the answer.
* Award 1 mark for the criterion, if the answer has a sentence of at least 10 words of explanation ONLY focussing on that phrase AND also exact match.
* Award 0.5 mark for the criterion, if there is only a partial string match of that criterion in student answer.

**Example:**

1.
Question - Evaluate the security robustness of blockchain networks against potential vulnerabilities such as 51% attacks and double-spending.
Answer - The analysis evaluates the security robustness of blockchain networks, with a focus on vulnerabilities such as 51% attacks and double-spending.

Marks = 0 (Answer is a simple rephrasing of the question)


**Explanation:**

Provide a **detailed and objective explanation** of the:
* Awarded score based on the rubric.
* **Reasoning** for assigning points or penalties for each criterion, referencing specifics from the rubric and student answer.

**Marks:**

Add the marks achieved for the answer for each criterion and provide the total mark in a separate line.

Student mark: Mark achieved from criterion1 + criterion2 + criterion3 + criterion4 + criterion5
[/INST]
""")

chain2 = LLMChain(prompt=prompt2, llm=llm_2)
response = chain2.invoke({"student_answer": student_answer, "rubric": rubric, "question": question})
inst_index = response['text'].find("[/INST]")
res = response['text'][inst_index + len("[/INST]"):]
res

"\n\n**Awarded score based on the rubric:**\n\n1. Analyze genomic data to identify genetic markers: The student answer explains that bioinformatics methods are used to analyze genomic data and uncover genetic links to complex diseases, which leads to the identification of genetic markers. This explanation is an exact match with criterion 1, and it is more than 10 words long. Therefore, the student receives 1 mark for this criterion.\n2. Pinpoint genetic variations associated with diseases: The student answer explains that genetic variations are identified by comparing the genetic makeup of individuals with and without the disease. This explanation is an exact match with criterion 2, and it is more than 10 words long. Therefore, the student receives 1 mark for this criterion.\n3. Develop personalized medicine approaches: The student answer explains that personalized medicine approaches are developed based on an individual's unique genetic makeup. This explanation is an exact match with 

#### Answer evaluation (without rubric) - provides similarity score

In [ ]:
prompt1 = ChatPromptTemplate.from_template("""[INST] You are a helpful, respectful and a good assistant. You will follow my words. Answer the question with the help of rubrics (rubrics contains the important points that should be in the answer). Do not mention anything about marks in the answer. Generate only the answer, not anything else. Answer only in 200 words.

Rubrics:
{rubrics}
------------- rubric ends here --------------

Question: {question} [/INST]""")
chain1 = LLMChain(prompt=prompt1, llm=llm_2)
response = chain1.invoke({"rubrics": rubric, "question": question})
inst_index = response['text'].find("[/INST]")
id_ans = response['text'][inst_index + len("[/INST]"):]

In [ ]:
ideal_answer = id_ans
prompt = ChatPromptTemplate.from_template("""[INST] You are a helpful, respectful and a very brutally honest assistant. You shall not generate any new answer as student answer, no matter what. Give me a similarity score from 0.000 to 1.000 (3 decimal positions), based on semantic similarity, content similarity and the level of explanation between the below given student answer and ideal answer, and then an explanation for it, Make sure that you make your evaluation strict:

    IDEAL ANSWER: {ideal_answer}
    STUDENT ANSWER: {student_answer}
    [/INST]""")
chain = LLMChain(prompt=prompt, llm=llm_2)
response = chain.invoke({"student_answer":student_answer, "ideal_answer":ideal_answer})
inst_index = response['text'].find("[/INST]")
res = response['text'][inst_index + len("[/INST]"):]
res

" Similarity Score: 0.985\n\nExplanation: The student answer and the ideal answer share a high level of semantic similarity as they both discuss the application of bioinformatics methods to analyze genomic data and identify genetic markers associated with complex diseases. The student answer mentions the use of various tools like sequence alignment, genome mapping, and variant analysis, which aligns with the ideal answer's mention of computational tools and statistical analyses. The student answer also mentions the identification of genetic variations associated with increased risk or response to treatment, which is similar to the ideal answer's discussion of genetic markers and personalized medicine approaches. The student answer also correctly identifies the ultimate goal of personalized medicine, which is to improve treatment effectiveness and patient well-being. The minor differences between the two answers include the student answer's more concise and simplified language and the i